<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# PRISM downsampling and reprojection

_by Joe Hamman (CarbonPlan), February 16, 2021_

This notebook downsamples and reprojects PRISM 300m climate normal rasters
stored in Cloud Optimized GeoTIFF into 4000m GeoTIFFs.

**Inputs:**

- COG outputs from `01_prism_to_cogs.ipynb`

**Outputs:**

- COG outputs after downsampling and reprojection

**Notes:**

- Source CRS and projection extent come from NLCD


In [ ]:
import os
from carbonplan_data.utils import projections, setup
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
import rasterio
import fsspec

dst_profile = cog_profiles.get("deflate")

workdir, upload = setup("joe")

In [ ]:
fs = fsspec.get_filesystem_class("gs")()

In [ ]:
resampling = "average"
resolution = 4000

for region in ["conus", "ak"]:
    crs, extent = projections("albers", region)

    for var in ["ppt", "tmean"]:

        print(region, var)

        source = f"carbonplan-data/raw/prism/normals/{region}/800m/{var}.tif"
        cloud_target = (
            f"processed/prism/normals/{region}/{resolution}m/{var}.tif"
        )

        local_source = "./local_source.tif"
        fs.get_file(source, local_source)

        cmd = (
            "gdalwarp",
            "-t_srs",
            f"'{crs}'",
            "-te",
            extent,
            "-tr",
            f"{resolution} {resolution}",
            "-r",
            resampling,
            local_source,
            "./raster.tif",
        )
        print(" ".join(cmd))
        os.system(" ".join(cmd))

        cog_translate("./raster.tif", "./raster.tif", dst_profile)
        upload("./raster.tif", cloud_target)

        os.remove(
            "raster.tif",
        )
        os.remove(
            "local_source.tif",
        )